## Introduction


This notebook explores how to wrap the chatbot in a gradio chat interface

###  Setting up paths and loading vector store

In [22]:
# === 0) Imports & manifest ===
from pathlib import Path
import yaml, json
from typing import Optional, List, Dict
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.tools import tool
from langchain.schema import Document

# If you want the prebuilt ReAct agent:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

# If you prefer to also expose tools directly on the LLM:
from langchain.chat_models import init_chat_model

# --- Set your pack root + manifest ---
ROOT = Path("/Users/ktejwani/Personal CS Projects/Summer 2025/Offline AI Kiosk/Offline-AI-Kiosk/hurricane_disaster_response_pack")
MANIFEST = ROOT / "manifest.yaml"

with open(MANIFEST, "r", encoding="utf-8") as f:
    manifest = yaml.safe_load(f)

# --- Resolve FAISS paths from manifest ---
faiss_dir_text = ROOT / manifest["precomputed_indices"]["text"]["faiss"]["dir"]
faiss_dir_image = ROOT / manifest["precomputed_indices"]["images"]["faiss"]["dir"]
# --- Create embeddings *matching the store* ---
embed_model_name = manifest["embedding_config"]["text"]["model"]     #same for text and image
emb = OllamaEmbeddings(model=embed_model_name)

# --- Load FAISS store + retriever ---
text_vs = FAISS.load_local(str(faiss_dir_text), emb, allow_dangerous_deserialization=True)
text_retriever = text_vs.as_retriever(search_kwargs={"k": 4})  # default k; tool will override if provided

image_vs = FAISS.load_local(str(faiss_dir_image), emb, allow_dangerous_deserialization=True)
# results = image_vs.similarity_search_with_score(query, k=4)






### Defining helper and context() Tool  

In [23]:
# === 1) Helpers ===
def format_chunk(doc: Document, max_chars: int = 400) -> Dict:
    """Return a dict with compact text + key metadata for prompting & audit."""
    txt = doc.page_content.strip()
    if len(txt) > max_chars:
        txt = txt[:max_chars].rstrip() + " …"
    m = doc.metadata
    return {
        "id": m.get("chunk_id"),
        "topic_id": m.get("topic_id"),
        "file_id": m.get("file_id"),
        "locale": m.get("locale"),
        "path": m.get("path"),
        "citations": [c.get("title", "") for c in m.get("citations", [])],
        "text": txt
    }

def format_context_block(chunks: List[Dict]) -> str:
    """Human/LLM-friendly context block the agent can drop into its reasoning."""
    lines = []
    lines.append("### Retrieved Context (use only what is relevant)")
    for i, c in enumerate(chunks, 1):
        cite_str = "; ".join([t for t in c["citations"] if t]) or "—"
        head = f"[{i}] {c['topic_id']} · {c['file_id']} · {c['locale']} · {c['path']}"
        lines.append(head)
        lines.append(c["text"])
        lines.append(f"Source(s): {cite_str}")
        lines.append("")  # blank line
    return "\n".join(lines).strip()


# === 2) The @tool: context() ===
@tool
def context(
    query: str,
    k: int = 4,
    topic_id: Optional[str] = None,
    locale: Optional[str] = None
) -> dict:
    """
    PURPOSE
    Retrieve Pack knowledge for the user's query. 
    TRIGGERS — YOU MUST CALL THIS TOOL BEFORE ANSWERING OR REFUSING IF:
      1) The query involves local geography or place-specific info:
         - “nearby”, “closest”, “where is…”, shelters/clinics/transport, checkpoints, routes, hours, hazards
      2) The query is wellbeing/safety-critical or time-sensitive:
         - first aid, symptoms, medications, exposure, heat/cold, water/food safety, wounds, evacuation, flooding, snakebite, pesticides, hazardous materials
      3) The query is high-uncertainty and a mistake could harm the user.

    Do NOT refuse until you have called this tool at least once.

    Args:
        query: Natural-language question or keywords.
        k: Top-k chunks to return (default 4).
        topic_id: Optional manifest topic filter (e.g. "bleed-control").
        locale: Optional locale (e.g. "en", "hi_en").

    Returns:
        {
          "query": str,
          "k": int,
          "filters": {"topic_id":..., "locale":...},
          "context_block": str,   # pasteable summary of chunks
          "chunks": [ {id, topic_id, file_id, path, locale, citations[], text}, ... ]
        }

    USAGE NOTES
    - Use retrieved content to ground your answer 
    - If nothing relevant is found, say so and offer next-best actions present in the Pack.
    """
    # Build a metadata filter if provided
    _filter = {}
    # if topic_id:
    #     _filter["topic_id"] = topic_id
    # if locale:
    #     _filter["locale"] = locale

    # Run retrieval (override k)
    local_ret = text_vs.as_retriever(search_kwargs={"k": k})
    hits: List[Document] = local_ret.invoke(query) if not _filter else local_ret.invoke(query, filter=_filter)

    formatted = [format_chunk(d) for d in hits]
    ctx_block = format_context_block(formatted)
    return {
        "query": query,
        "k": k,
        "filters": _filter,
        "context_block": ctx_block,
        "chunks": formatted
    }


### Math Tools

In [24]:
@tool
def add(a: float, b: float) -> float:
    """Add two numbers.
    
    Args:
        a: First float
        b: Second float
    """
    return a + b

@tool
def subtract(a: float, b: float) -> float:
    """Subtract first number by second number.

    Args:
        a: First float
        b: Second float
    """
    return a - b

@tool
def multiply(a: float, b: float) -> float:
    """Multiply two numbers.

    Args:
        a: First float
        b: Second float
    """
    return a * b

@tool
def divide(a : float, b: float) -> float:
    """Divide first number by second number.
    
    Args:
        a: First float
        b: Second float
    """
    if b == 0:
        return 0
    else:
        return a/b


### Defining knowledgeMeta() tool

In [25]:
from langchain.tools import tool
from pathlib import Path
import yaml
from typing import Optional

@tool
def knowledgeMeta(pack_dir: Optional[str] = None) -> dict:
    """
    Read a knowledge pack manifest and return metadata for trust and recency. 

    Args:
      pack_dir: Absolute or relative path to the pack folder (containing manifest.yaml).
                If omitted, uses the default ROOT pack path.

    Returns:
      {
        "name": str,
        "version": str,
        "date": str,
        "locales": [..],
        "topics_count": int,
        "manifest_path": str
      }

    """
    # default to your earlier ROOT if not provided
    base = Path(pack_dir) if pack_dir else ROOT
    manifest_path = base / "manifest.yaml"
    if not manifest_path.exists():
        return {"error": f"manifest.yaml not found at {manifest_path}"}

    with open(manifest_path, "r", encoding="utf-8") as f:
        m = yaml.safe_load(f)

    name = m.get("name", str(base.name))
    version = m.get("version", "unknown")
    date = m.get("date", "unknown")
    locales = m.get("locales", [])
    topics = m.get("index_of_topics", []) or []
    return {
        "name": name,
        "version": version,
        "date": date,
        "locales": locales,
        "topics_count": len(topics),
        "manifest_path": str(manifest_path)
    }


### getImage tool

In [26]:
from langchain_core.tools import tool
from IPython.display import Image, display
from pathlib import Path
from typing import Dict, Any

HIGH_SCORE_THRESHOLD = 0.55  # tune as needed

@tool
def getImage(query: str) -> Dict[str, Any]:
    """
    PURPOSE
    Retrieve a single high-confidence Pack image for the query. DO NOT DISPLAY IMAGE, IT IS DONE SO AUTOMATICALLY AT THE TOP OF YOUR MESSAGE.

    TRIGGERS — YOU MUST CALL THIS TOOL WHEN:
      1) The user asks to "show" or "see" something (e.g., “show me the Heimlich position”),
      2) The user requests a diagram or visual (diagram / illustrate / picture / visual / map),
      3) A visual guide would materially improve a physical technique (CPR posture, tourniquet placement, splinting, boiling water, wound cleaning, snakebite immobilization).

    If no suitable image is found, return NO_IMAGE and proceed with clear step-by-step text (and cite “context” if used).

    Returns:
        {
          "status": "OK" | "NO_IMAGE",
          "version": str,
          "date": str,
          "locales": [str],
          "pack_name": str,
          "image_path": str,      # absolute or pack-relative path
          "score": float | None,
          "citations": list       # e.g., [{"title": "...", ...}]
        }
    """
    q = (query or "").strip()
    pack_name    = manifest.get("name", "")
    pack_ver     = manifest.get("version", "")
    pack_date    = manifest.get("date", "")
    pack_locales = manifest.get("locales", [])

    if not q:
        return {
            "status": "NO_IMAGE",
            "version": pack_ver,
            "date": pack_date,
            "locales": pack_locales,
            "pack_name": pack_name,
            "image_path": "",
            "score": None,
            "citations": []
        }

    # Use similarity_search_with_score to get confidence scores
    foundImage = False
    minScore = 0.32
    finalDoc  = Document(page_content="")
    results = image_vs.similarity_search_with_score(query, k=4)
    finScore = 0
    from IPython.display import Image, display

    for i, (d, score) in enumerate(results, 1):
        
        if score >= minScore:
            finScore = score
            finalDoc = d
            foundImage = True
            break
    

    if not foundImage:
        print("NOT FOUND")
        return {
            "status": "NO_IMAGE",
            "version": pack_ver,
            "date": pack_date,
            "locales": pack_locales,
            "pack_name": pack_name,
            "image_path": "",
            "score": None,
            "citations": []
        }
    else:
        print("Found FOUND")
        img_path = ROOT / finalDoc.metadata['path']
        # try:
        #     display(Image(filename=img_path))
        # except Exception:
        #     pass
            
        return {
            "status": "OK",
            "version": pack_ver,
            "date": pack_date,
            "locales": pack_locales,
            "pack_name": pack_name,
            "image_path": str(img_path),
            "score": float(finScore),
            "citations": finalDoc.metadata.get("citations", [])
        }
        


### List of Available Tools

In [27]:
# Keep a Tools list for whichever orchestration you choose:
TOOLS = [context,add,multiply,subtract,divide,knowledgeMeta, getImage]

### Setting Up LLM

In [28]:
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent
# from langchain_openai import ChatOpenAI
from gradio import ChatMessage
import gradio as gr
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder



from langchain.chat_models import init_chat_model

lightModel = init_chat_model(
    model="ollama:llama3.1",       
    temperature=0.2, # lower = more deterministic
    streaming = True  
)

heavyModel = init_chat_model(
    model="ollama:gpt-oss:20b",       
    temperature=0.2  # lower = more deterministic
)

### Using gradio with langchain



This is a simple general-purpose chatbot built on top of LangChain and Gradio.


In [29]:
from langchain.schema import AIMessage, HumanMessage  
import gradio as gr

lightModel = init_chat_model(
    model="ollama:llama3.1",       
    temperature=0.2  # lower = more deterministic
)

def predict(message, history):
    history_langchain_format = []
    for msg in history:
        if msg['role'] == "user":
            history_langchain_format.append(HumanMessage(content=msg['content']))
        elif msg['role'] == "assistant":
            history_langchain_format.append(AIMessage(content=msg['content']))
    history_langchain_format.append(HumanMessage(content=message))
    llm_response = lightModel.invoke(history_langchain_format)
    return llm_response.content

demo = gr.ChatInterface(
    predict,
    type="messages"
)

demo.launch()


* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.


### Working UI with tool calling and sources

Images is next 

In [30]:

tools = TOOLS


from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent
# from langchain_openai import ChatOpenAI
from gradio import ChatMessage
import gradio as gr
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools.render import render_text_description

# Render tool descriptions and names
tool_desc = render_text_description(TOOLS)         # Render the tool name and description in plain text.
tool_names = ", ".join([t.name for t in TOOLS])    # exact callable names

# Build agent prompt
prompt = ChatPromptTemplate.from_messages([
    ("system",
     """You are Beacon, a helpful assistant that answers using the active Knowledge Pack (domain- and locale-specific, offline-first).

HARD RULES — TOOL USE
1) You MUST call the tool named “context” BEFORE answering or refusing if:
   • The user asks about local geography, “near me/nearby/closest”, directions, hours, routes, checkpoints, shelters, clinics, water points, transport, or place-specific availability, OR
   • The user’s request affects health, safety, or wellbeing (first aid, symptoms, medications, exposure, heat/cold, water safety, food safety, wound care, evacuation decisions, flooding, snakebite, pesticides, hazardous materials), OR
   • The query is high-stakes, time-sensitive, or ambiguous in a way that could impact safety.
   → Do not refuse UNTIL you have called “context”.

2) You MUST call the tool named “getImage” when:
   • The user says “show me …”, “what does … look like?”, “diagram”, “visual”, “illustrate”, OR
   • A visual guide would materially improve understanding for a physical technique (e.g., CPR posture, Heimlich, tourniquet placement, splinting, water boiling steps, winding a bandage).
   If “getImage” returns NO_IMAGE, continue with clear, step-by-step text and cite sources from “context” if available.

3) Tool names must match exactly from {tool_names}. Prefer the smallest sufficient k. If a Pack locale or topic is clear, pass it.

4) Citations: When “context” is used, include a short “Sources” line drawn from its returned citations when you give final guidance.

5) Refusals: Only refuse after calling “context” if (a) the Pack lacks relevant guidance or (b) the request is outside your guardrails. In refusals, suggest the nearest safe alternative or escalation path if the Pack provides one.

OUTPUT STYLE
• Be concise, stepwise, and actionable. If life/safety-critical, front-load DOs and DON’Ts and emphasize time-critical steps.
• If the Pack is ambiguous, state assumptions briefly and continue.
• NEVER hallucinate locations or phone numbers—use “context”. If none found, say so and offer next-best actions present in the Pack.

You MUST use the following tools at least once if the above rules apply:
{tools}
(Always call tools by exact name from: {tool_names}. If no rule triggers, you may answer directly.)
"""),
    MessagesPlaceholder("chat_history", optional=True),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),
]).partial(tools=tool_desc, tool_names=tool_names)

agent = create_tool_calling_agent(
    llm=heavyModel,
    tools=tools,
    prompt=prompt,
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,        # helpful while wiring things up
).with_config({"run_name": "Agent"})


### DEV UI (OLD)

In [31]:
from typing import Dict, Any, List

def _format_sources_md(obs: Dict[str, Any]) -> str:
    """
    Render Sources from manifest-style citations only.
    Accepts tool observations shaped like:
      - getImage: { citations: [{title, id, url, license}, ...] }
      - context:  { chunks: [{..., citations:[{title,id,url,license}, ...]} , ...] }

    We ignore paths / file_ids and only show (title, url, license).
    """
    if not isinstance(obs, dict):
        return ""

    # Collect citation dicts from:
    #  A) top-level obs["citations"]
    #  B) each chunk's ["citations"] (if present)
    collected: List[Dict[str, Any]] = []

    top_cites = obs.get("citations", [])
    if isinstance(top_cites, list):
        for c in top_cites:
            if isinstance(c, dict):
                collected.append(c)

    chunks = obs.get("chunks", [])
    if isinstance(chunks, list):
        for ch in chunks:
            if not isinstance(ch, dict):
                continue
            ch_cites = ch.get("citations", [])
            if isinstance(ch_cites, list):
                for c in ch_cites:
                    if isinstance(c, dict):
                        collected.append(c)

    # Deduplicate by (url or title), then format
    seen = set()
    lines: List[str] = []
    for c in collected:
        title = (c.get("title") or c.get("id") or "Source").strip()
        url   = (c.get("url") or "").strip()
        lic   = (c.get("license") or "—").strip()

        key = (url or title).lower()
        if key in seen:
            continue
        seen.add(key)

        if url:
            lines.append(f"- [{title}]({url}) · {lic}")
        else:
            lines.append(f"- {title} · {lic}")

    if not lines:
        return ""

    return "**Sources**\n" + "\n".join(lines)


def _maybe_render_image_from_obs(obs) -> ChatMessage | None:
    """
    Accepts a tool observation dict and returns a ChatMessage with an image
    if it finds a usable image path/url. Otherwise returns None.
    """
    if not isinstance(obs, dict):
        return None

    # Common keys your chain/logs might produce
    path = (
        obs.get("image_path")
        or obs.get("path")
        or obs.get("file_path")
        or obs.get("local_path")
        or None
    )
    url = obs.get("image_url") or obs.get("url")

    # Prefer local path if available; Gradio will serve files inside the app folder.
    media = path or url
    if not media:
        return None

    # Option A: return a component (nice rendering)
    return ChatMessage(role="assistant", content=gr.Image(value=media))


# --- Async streaming handler for Gradio Chatbot(type="messages") ---
async def interact_with_langchain_agent(user_text, history):
    # history is a list[ChatMessage]
    history.append(ChatMessage(role="user", content=user_text))
    yield history

    # Stream agent steps & final output
    async for chunk in agent_executor.astream({"input": user_text}):
        if "steps" in chunk:
            for step in chunk["steps"]:
                history.append(ChatMessage(
                    role="assistant",
                    content=step.action.log,
                    metadata={"title": f"🛠️ Used tool {step.action.tool}"}
                ))


                obs = getattr(step, "observation", None)
                if obs is not None:
                    # Try rendering an image if present (unchanged)
                    img_msg = _maybe_render_image_from_obs(obs)
                    if img_msg is not None:
                        history.append(img_msg)
                        yield history

                    # NEW: one merged sources renderer for both getImage + context
                    sources_md = _format_sources_md(obs)  # ← no second argument now
                    print(sources_md)
                    if sources_md:
                        history.append(ChatMessage(role="assistant", content=sources_md))
                        yield history

                yield history
                #     else:
                #         # For other tools you can also render their observations if you want
                #         pass


                # yield history
        if "output" in chunk:
            history.append(ChatMessage(role="assistant", content=chunk["output"]))
            yield history


# --- Gradio UI ---
with gr.Blocks() as demo:
    gr.Markdown("# Beacon - Knowledge Agent (Pinellas County, FL)")

    chatbot = gr.Chatbot(
        type="messages",
        label="Agent",
        avatar_images=(None, "/Users/ktejwani/Personal CS Projects/Summer 2025/Offline AI Kiosk/Offline-AI-Kiosk/avatar.png"),
        height=650,
    )
    textbox = gr.Textbox(lines=1, label="Chat Message", placeholder="Ask something…")
    # Clear the textbox after submit so it feels chatty
    def _clear_now(_msg, _chat):
        return gr.update(value="")

    # streaming submit
    textbox.submit(
        interact_with_langchain_agent,
        inputs=[textbox, chatbot],
        outputs=[chatbot],
    )

    # instant clear
    textbox.submit(
        _clear_now,
        inputs=[textbox, chatbot],   # must match the event’s inputs (2 args)
        outputs=[textbox],
        queue=False,
)



demo.queue().launch()   # queue() is recommended for async callbacks


* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.




> Entering new Agent chain...

Invoking: `context` with `{'k': 4, 'locale': 'en', 'query': 'evacuation points'}`


{'query': 'evacuation points', 'k': 4, 'filters': {}, 'context_block': '### Retrieved Context (use only what is relevant)\n[1] hurricane-readiness · all-hazard-guide · en · core/hurricane-readiness/en/All_Hazard_Guide-002.pdf\nBecause evacuations take time to ensure everyone can get to safety, they are called \nwell in advance of the storm. Evacuations have a beginning and end time. You must \nbe in a safe location by the end of the evacuation period, well before  \nthe storm surge and high winds arrive.\nEvacuation orders issued by Pinellas County are for the entire county, including  cities \nand unincorporated areas.\nList …\nSource(s): Pinellas County: All-Hazard Preparedness Guide (Liberty copy)\n\n[2] flooding-storm-safety · all-hazard-preparedness · en · core/flood-storm-safety/en/Pinellas-County-All-Hazard-Guide.pdf\nPAGE 8\n   ALL HAZARDS PREPAREDNESS GUIDE    A

### CURRENT UI NO TOOL CALLING

In [ ]:
import asyncio
from typing import Dict, Any, List

# --- citations: manifest-only (title, url, license) ---
def _format_sources_md(obs: Dict[str, Any]) -> str:
    if not isinstance(obs, dict):
        return ""
    collected: List[Dict[str, Any]] = []

    # top-level citations (e.g., getImage)
    top = obs.get("citations")
    if isinstance(top, list):
        for c in top:
            if isinstance(c, dict):
                collected.append(c)

    # chunk-level citations (e.g., context)
    chunks = obs.get("chunks")
    if isinstance(chunks, list):
        for ch in chunks:
            if not isinstance(ch, dict):
                continue
            ch_cites = ch.get("citations")
            if isinstance(ch_cites, list):
                for c in ch_cites:
                    if isinstance(c, dict):
                        collected.append(c)

    seen = set()
    lines: List[str] = []
    for c in collected:
        title = (c.get("title") or c.get("id") or "Source").strip()
        url   = (c.get("url") or "").strip()
        lic   = (c.get("license") or "—").strip()
        key = (url or title).lower()
        if key in seen:
            continue
        seen.add(key)
        if url:
            lines.append(f"- [{title}]({url}) · {lic}")
        else:
            lines.append(f"- {title} · {lic}")

    return "**Sources**\n" + "\n".join(lines) if lines else ""


def _maybe_render_image_from_obs(obs) -> ChatMessage | None:
    if not isinstance(obs, dict):
        return None
    path = (
        obs.get("image_path")
        or obs.get("path")
        or obs.get("file_path")
        or obs.get("local_path")
        or None
    )
    url = obs.get("image_url") or obs.get("url")
    media = path or url
    if not media:
        return None
    return ChatMessage(role="assistant", content=gr.Image(value=media))


# --- Async streaming handler for Gradio Chatbot(type="messages") ---
async def interact_with_langchain_agent(user_text, history):
    """
    Streams a conversation turn:
      - append user msg
      - show ⏳ placeholder
      - stream agent tool results (image + Sources only)
      - stream final answer
    Robust to exceptions; will surface errors in-chat and close cleanly.
    """
    # 1) user message
    history.append(ChatMessage(role="user", content=user_text))
    yield history

    # 2) thinking placeholder
    thinking_msg = ChatMessage(role="assistant", content="⏳ Thinking...")
    history.append(thinking_msg)
    yield history

    try:
        # 3) stream agent
        async for chunk in agent_executor.astream({"input": user_text}):

            # remove placeholder on first real activity
            if thinking_msg in history:
                try:
                    history.remove(thinking_msg)
                except ValueError:
                    pass  # already removed elsewhere

            # show ONLY user-facing results from tools
            if "steps" in chunk:
                for step in chunk["steps"]:
                    # DO NOT append step.action.log (keeps tool calls hidden)
                    obs = getattr(step, "observation", None)
                    if not isinstance(obs, dict):
                        continue

                    # image
                    img_msg = _maybe_render_image_from_obs(obs)
                    if img_msg is not None:
                        history.append(img_msg)
                        yield history

                    # sources
                    sources_md = _format_sources_md(obs)
                    if sources_md:
                        history.append(ChatMessage(role="assistant", content=sources_md))
                        yield history

                # yield after processing this chunk
                yield history

            # final assistant output
            if "output" in chunk:
                history.append(ChatMessage(role="assistant", content=chunk["output"]))
                yield history

        # finished normally — ensure placeholder is gone
        if thinking_msg in history:
            try:
                history.remove(thinking_msg)
            except ValueError:
                pass

    except Exception as e:
        # surface the real error so you can see it instead of a vague aclose warning
        if thinking_msg in history:
            try:
                history.remove(thinking_msg)
            except ValueError:
                pass
        history.append(ChatMessage(role="assistant", content=f"⚠️ Error: {e}"))
        yield history

    finally:
        # give Gradio a tick to close the async generator cleanly
        await asyncio.sleep(0)
        return


# --- Gradio UI ---
with gr.Blocks() as demo:
    gr.Markdown("# Beacon - Knowledge Agent (Village Bihar, India)")

    chatbot = gr.Chatbot(
        type="messages",
        label="Agent",
        avatar_images=(None, "/Users/ktejwani/Personal CS Projects/Summer 2025/Offline AI Kiosk/Offline-AI-Kiosk/avatar.png"),
        height=650,
    )
    textbox = gr.Textbox(lines=1, label="Chat Message", placeholder="Ask something…")

    # Clear the textbox after submit so it feels chatty
    def _clear_now(_msg, _chat):
        return gr.update(value="")

    # streaming submit
    textbox.submit(
        interact_with_langchain_agent,
        inputs=[textbox, chatbot],
        outputs=[chatbot],
    )

    # instant clear
    textbox.submit(
        _clear_now,
        inputs=[textbox, chatbot],   # must match the event’s inputs (2 args)
        outputs=[textbox],
        queue=False,
    )

demo.queue().launch()  # queue() is recommended for async callbacks


* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.




> Entering new Agent chain...
Hello! How can I help you today?

> Finished chain.
